In [1]:
import plotly.express as px
import numpy as np
import scipy as sp
import cellpose
import pandas as pd
import tifffile
import glob
from cellpose import models
import napari
%gui qt5
import skimage.measure as measure
import plotly.graph_objs as go
import scipy.stats as stats

In [2]:
viewer = napari.Viewer()

In [3]:
DAPI_channel = 0
piwi_channel = 1
porcna_channel = 2
H3P_channel = 3

# Do cellpose to find nucleii, save

We will bin together 4 slices at a time, taking the mean for the fluorescence and taking the 2nd slice for nucleii segmentation

Only need to run this if have not already run it.

The tif files should be the raw files but with the background subtracted.

In [4]:
model_cyto = models.Cellpose(gpu=True, model_type='cyto')

In [5]:
fnames = glob.glob('./*/*.tif')

In [6]:
for i,f in enumerate(fnames):
    print(i/len(fnames)*100)
    print(f)
    img = tifffile.imread(f)
    #img[:,0] = np.roll(img[:,0], 3, axis=0)
    #img[:,1] = np.roll(img[:,1], 2, axis=0)
    #img[:,2] = np.roll(img[:,2], 1, axis=0)
    if (len(img.shape)>4):
        img = img.mean(axis=0)
    
    new_frames = int(np.floor(img.shape[0]/6))
    
    reshaped_img = img[0:(new_frames*6),:,:,:].reshape([new_frames,6,4,2044,2048])

    flattened_img = np.mean(reshaped_img, axis=1)
    flattened_img[:,DAPI_channel,:,:] = np.max(reshaped_img[:,[2,3],DAPI_channel,:,:], axis=1)

    lst = [im for im in flattened_img[:,DAPI_channel,:,:]]
    nucleii = model_cyto.eval(lst, channels=[0,0], diameter=50, flow_threshold=0.7,
                           normalize=True, resample=False)[0]
    nucleii = np.array(nucleii)

    flattened_img = np.append(flattened_img, nucleii[:,np.newaxis,:], axis=1)
    tifffile.imwrite(f+'f', flattened_img.astype(np.uint16), imagej=True, metadata={'axes':'ZCYX'})

0.0
.\mmp1_piwi_h3p\intact1_piwi488_mmp1568_h3p647_40x.tif
3.8461538461538463
.\mmp1_piwi_h3p\intact2_piwi488_mmp1568_h3p647_40x001.tif
7.6923076923076925
.\mmp1_piwi_h3p\intact3_piwi488_mmp1568_h3p647_40x.tif
11.538461538461538
.\mmp1_piwi_h3p\intact4_piwi488_mmp1568_h3p647_40x001.tif
15.384615384615385
.\mmp1_piwi_h3p\intact5_piwi488_mmp1568_h3p647_40x002.tif
19.230769230769234
.\porcn_piwi_h3p\48hpa1_piwi488_porcn568_h3p647_40x030.tif
23.076923076923077
.\porcn_piwi_h3p\48hpa2_piwi488_porcn568_h3p647_40x031.tif
26.923076923076923
.\porcn_piwi_h3p\48hpa3_piwi488_porcn568_h3p647_40x032.tif
30.76923076923077
.\porcn_piwi_h3p\48hpa4_piwi488_porcn568_h3p647_40x033.tif
34.61538461538461
.\porcn_piwi_h3p\48hpa5_piwi488_porcn568_h3p647_40x034.tif
38.46153846153847
.\porcn_piwi_h3p\6hpa1_piwi488_porcn568_h3p647_40x025.tif
42.30769230769231
.\porcn_piwi_h3p\6hpa2_piwi488_porcn568_h3p647_40x026.tif
46.15384615384615
.\porcn_piwi_h3p\6hpa3_piwi488_porcn568_h3p647_40x027.tif
50.0
.\porcn_piwi_h3

# Quantify the labeled files

In [6]:
fnames = glob.glob('./porcn_piwi_h3p/*.tiff')

In [7]:
def get_intensities(imgs, labels):
    lst = []
    for im, lab in zip(imgs, labels):
        tlst = measure.regionprops(lab, im)
        [lst.append(t['intensity_mean']) for t in tlst]
    return np.array(lst)

In [8]:
def get_areas(labels):
    xylst = []
    zlst = []
    alst = []
    for slc, lab in enumerate(labels):
        tlst = measure.regionprops(lab, lab)
        [xylst.append(t['centroid']) for t in tlst]
        [alst.append(t['area']) for t in tlst]
        [zlst.append(slc) for t in tlst]
    return np.array(xylst), np.array(alst), np.array(zlst)

In [9]:
def filter_objects(mask, size):
    rtn = []
    for m in mask:
        labs = sp.ndimage.label(m)[0]
        props = measure.regionprops(labs, labs)
        lst = np.array([t['area'] for t in props])
        lst = np.append([0], lst)
        lst = lst>size
        size_img = lst[labs]
        rtn.append(size_img)
    return np.array(rtn)


In [10]:
def get_distances(edt, labels, show=False):
    return get_intensities(edt, labels)

In [11]:
def get_data(f, show=False):
    img = tifffile.imread(f)
    centroids, areas, slices = get_areas(img[:,4])
    c0 = get_intensities(img[:,0], img[:,4])
    c1 = get_intensities(img[:,1], img[:,4])
    c2 = get_intensities(img[:,2], img[:,4])
    c3 = get_intensities(img[:,3], img[:,4])
    if show:
        viewer.add_image(img, channel_axis=1)
    edt = tifffile.imread(f[0:-4]+'/Img.tiff')
    distances = get_distances(edt, img[:,4], show=show)
    cdf = pd.DataFrame({'Area':areas, 'X':centroids[:,0], 'Y':centroids[:,1], 'Z':slices, 'C0':c0, 'C1':c1,
                       'C2':c2, 'C3':c3, 'Distance':distances})
    cdf['File'] = f
    return cdf

In [10]:
df = pd.DataFrame()
for f in fnames:
    print(f)
    cdf = get_data(f)
    df = pd.concat([df,cdf])

./porcn_piwi_h3p\48hpa1_piwi488_porcn568_h3p647_40x030.tiff
./porcn_piwi_h3p\48hpa2_piwi488_porcn568_h3p647_40x031.tiff
./porcn_piwi_h3p\48hpa3_piwi488_porcn568_h3p647_40x032.tiff
./porcn_piwi_h3p\48hpa4_piwi488_porcn568_h3p647_40x033.tiff
./porcn_piwi_h3p\48hpa5_piwi488_porcn568_h3p647_40x034.tiff
./porcn_piwi_h3p\6hpa1_piwi488_porcn568_h3p647_40x025.tiff
./porcn_piwi_h3p\6hpa2_piwi488_porcn568_h3p647_40x026.tiff
./porcn_piwi_h3p\6hpa3_piwi488_porcn568_h3p647_40x027.tiff
./porcn_piwi_h3p\6hpa4_piwi488_porcn568_h3p647_40x028.tiff
./porcn_piwi_h3p\6hpa5_piwi488_porcn568_h3p647_40x029.tiff
./porcn_piwi_h3p\intact1a_piwi488_porcn568_h3p647_40x013.tiff
./porcn_piwi_h3p\intact1b_piwi488_porcn568_h3p647_40x014.tiff
./porcn_piwi_h3p\intact2a_piwi488_porcn568_h3p647_40x015.tiff
./porcn_piwi_h3p\intact3a_piwi488_porcn568_h3p647_40x017.tiff
./porcn_piwi_h3p\intact3b_piwi488_porcn568_h3p647_40x018.tiff
./porcn_piwi_h3p\intact4a_piwi488_porcn568_h3p647_40x019.tiff
./porcn_piwi_h3p\intact4b_piwi488

In [12]:
df.to_csv('Quantification_porcna.csv')

# Analyze results

In [12]:
df = pd.read_csv('Quantification_porcna.csv')

In [13]:
df['Time'] = df['File'].str.split('\\').str[1].str.split('_').str[0].str[0:-2]

In [11]:
#df['Label'] = df['File'].str.split('_').str[0]
#df['Time'] = df['File'].str.split('_').str[3]
#df['ID'] = df['File'].str.split('_').str[4]
#df['Side'] = df['File'].str.split('_').str[5]

#df.loc[df['Label']=='No','Time'] = df.loc[df['Label']=='No','File'].str.split('_').str[2]
#df.loc[df['Label']=='No','ID'] = df.loc[df['Label']=='No','File'].str.split('_').str[3]
#df.loc[df['Label']=='No','Side'] = df.loc[df['Label']=='No','File'].str.split('_').str[4]



In [14]:
def ptile(arg):
    return np.percentile(arg, 30)

In [15]:
pt = df.pivot_table(index=['File','Z'], values=['C0','C1','C2', 'C3'], aggfunc=ptile)
df = df.merge(pt.reset_index().rename(columns={'C0':'C0ptile', 'C1':'C1ptile', 'C2':'C2ptile', 'C3':'C3ptile'}), on=['File','Z'])
df['C0raw'] = df['C0']
df['C2raw'] = df['C2']
df['C0'] = df['C0']/df['C0ptile']
df['C1'] = df['C1']/df['C1ptile']
df['C2'] = df['C2']/df['C2ptile']
df['C3'] = df['C3']/df['C3ptile']

In [16]:
#pt = df.pivot_table(index=['Label', 'Time', 'ID','Z'], values=['C0','C1','C2', 'C3'], aggfunc=ptile)
#df = df.merge(pt.reset_index().rename(columns={'C0':'C0ptile', 'C1':'C1ptile', 'C2':'C2ptile', 'C3':'C3ptile'}), on=['Label',
#                                                                                                                     'Time', 'ID'
#                                                                                                                     ,'Z'])
#df['C0'] = df['C0']/df['C0ptile']
#df['C1'] = df['C1']/df['C1ptile']
#df['C2'] = df['C2']/df['C2ptile']
#df['C3'] = df['C3']/df['C3ptile']

In [17]:
df['L0'] = np.log(df['C0'])
df['L1'] = np.log(df['C1'])
df['L2'] = np.log(df['C2'])
df['L3'] = np.log(df['C3'])

In [18]:
df['Z'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56], dtype=int64)

In [25]:
visualize=False

In [26]:
if visualize:
    px.histogram(df[df['Z']<20], x='L1', animation_frame='File', facet_col='Z', facet_col_wrap=3, height=1200, range_x=[-1,2], range_y=[0,100])

In [27]:
if visualize:
    px.density_heatmap(df, x="L1", y="L3",  
                   nbinsx=50, nbinsy=50, height=1200, animation_frame='File',
                   color_continuous_scale=[(0, "blue"), (0.05, "green"), (0.4, 'yellow'), (1, "red")])
                  

# Look at data

### Threshold for piwi and markers

In [28]:
piwi_thresh = 1.2

H3p_thresh = 4.0

In [29]:
df['H3pPos'] = df['L3']>H3p_thresh
df['PiwiPos'] = df['L1']>piwi_thresh
df['DoublePos'] = df['H3pPos'] & df['PiwiPos']

df = df[df['Z']<25]

In [30]:
gdf = df.groupby(['Time', 'File']).agg({'H3pPos':np.mean, 'PiwiPos':np.mean, 'DoublePos':np.mean}).reset_index()
gdf['FractionDoublePos'] = gdf['DoublePos']/gdf['PiwiPos']

### Threshold based analysis

In [31]:

if visualize:
    f=go.FigureWidget(
        px.box(gdf, x='Time', y='PiwiPos', points='all',
            hover_data=['File'])
        )
     
    def click_fn(trace, points, state):
        
        if (len(points.point_inds)>0):
            file = f.data[points.trace_index]['customdata'][points.point_inds[-1]][0]
            print(file)
            img = tifffile.imread(file)[0:25]
            print(img.shape)
            viewer.layers.clear()
            viewer.add_image(img[:,0:4], channel_axis=1)
            viewer.add_labels(img[:,4])
            pts = (df[(df['File']==file) & (df['PiwiPos'])][['Z', 'X', 'Y']].to_numpy())
            viewer.add_points(pts, size=20, face_color='magenta')
            pts = (df[(df['File']==file) & (df['H3pPos'])][['Z', 'X', 'Y']].to_numpy())
            viewer.add_points(pts, size=20, face_color='red')
            viewer.layers[1].contrast_limits=[0,4000]
            edt = tifffile.imread(file[0:-4]+'/Img.tiff')[0:25]
            viewer.add_image(edt)
            viewer.layers[0].colormap='green'
            
            #viewer.layers[0].visible=False
            viewer.layers[2].visible=False
            viewer.layers[4].visible=False
            viewer.layers[3].colormap='gray'
            

    for a in range(0,len(f.data)):
        f.data[a].on_click(click_fn)

    f

## Distance analysis

In [32]:
if visualize:
    f = px.violin(df[df['PiwiPos']], x='Time', y='Distance',  color='DoublePos', title='Piwi positive cells, minimum distance to porcna region, not aggregated by image',
          hover_data=['File'])
    f.write_html('MinDistByCell_porcna.html')
    f

In [33]:
ddf = df[df['PiwiPos']].groupby(['File', 'DoublePos', 'Time']).agg({'Distance':np.median}).reset_index()

In [34]:
if visualize:
    f = px.box(ddf, color='DoublePos', x='Time', y='Distance', points='all', title='Piwi positive cells, minimum distance to porcna region, aggregated by image')
    f.write_html('MinDistByFile_porcna.html')
    f

If we look at ALL cells, not just piwi positive, do we see similar trends.  Ie, is this merely a reflection of there being more/less porcna in the image?

In [35]:
if visualize:
    dddf = df.groupby(['File', 'DoublePos', 'Time']).agg({'Distance':np.median}).reset_index()
    px.box(dddf, x='Time', y='Distance', points='all', title='Cells minimum distance to porcna region, aggregated by image')

It would appear that for whatever reason, the areas imaged had more gut in the intact

## Closeness analysis

In [36]:
df['Close'] = df['Distance']<20/1000000

In [37]:
ttdf = df.copy()
ttdf['Distance'] = ttdf['Distance']*1000000
ttdf.to_csv('AllCell_porcna.csv')

In [34]:
gdf = df.groupby(['Time', 'File', 'Close']).agg({'H3pPos':np.mean, 'PiwiPos':np.mean, 'DoublePos':np.mean}).reset_index()
gdf['FractionDoublePos'] = gdf['DoublePos']/gdf['PiwiPos']

### Looking at fraction of all cells, close vs far

In [38]:
if visualize:
    px.box(gdf, x='Time', y='PiwiPos', color='Close', points='all')

In [39]:
if visualize:
    px.box(gdf, x='Time', y='H3pPos', color='Close', points='all')

In [40]:
if visualize:
    px.box(gdf, x='Time', y='DoublePos', color='Close', points='all')

In [41]:
pdf = df[df['PiwiPos']].groupby(['Time', 'File', 'Close']).agg({'H3pPos':np.mean, 'PiwiPos':np.mean, 'DoublePos':np.mean}).reset_index()
pdf['FractionDoublePos'] = pdf['DoublePos']/pdf['PiwiPos']

In [42]:
if visualize:
    px.box(pdf, x='Time', y='DoublePos', color='Close', points='all')